In [37]:
# import neccessary libraries for EDA phase
import pandas as pd
import numpy as np 
import math
import matplotlib.pyplot as plt 
import seaborn as sns  

In [38]:
# create connection to database to load csv 
# FYI: I could've saved it to csv and loaded it via pd.read_csv but this project is to show my understand of ETL process

import sqlite3 
db_path = '/Users/juskiiii/Saftey-Risk-Analysis-1/database/Safety.db'
conn = sqlite3.connect(db_path)
site_exposure = pd.read_sql('SELECT * from int_site_week_hours;', conn)

In [39]:
site_exposure.head()

,site,fiscal_year,period_name,week_sum,air_incidents,hours,rate,period_over_period_pct_change,missing_flag_hours,employee_hours,contractor_hours,total_hours_worked
0,teb3,2025,2025 WK52,52,203,1651921.908,24.577433,-0.0027,1,NaN,NaN,NaN
1,teb3,2025,2025 WK51,51,201,1631332.059,24.642439,-0.0010,1,NaN,NaN,NaN
2,teb3,2025,2025 WK50,50,197,1597327.677,24.666198,-0.0027,0,36615.0551,0.0,36615.0551
3,teb3,2025,2025 WK49,49,193,1560715.121,24.732252,-0.0089,0,37833.4828,0.0,37833.4828
4,teb3,2025,2025 WK48,48,190,1522885.305,24.952634,0.0040,0,34153.6103,0.0,34153.6103


In [40]:
# merge features table for site exposure togther 

feat_site_exposure = pd.read_sql('SELECT * from feat_site_week_hours_risk', conn)

site_exposure = site_exposure.merge(feat_site_exposure, how= 'left', on = 'week_sum')

In [41]:
# remove unneccessary columns 

site_exposure.drop(columns= ['period_name', 'rate', 'period_over_period_pct_change',
                             'total_hours_worked', 'site_y', 'fiscal_year_y', 'air_incidents_y',
                             'hours', 'contractor_hours', 'employee_hours',], inplace = True)

In [42]:
# fix column label typos 
site_exposure.rename(columns = {'site_x' : 'site', 'fiscal_year_x': 'year', 
                               'air_incidents_x': 'all_incidents'}, inplace = True)

In [ ]:
# find where nan values exist 
def count_nan(df):            # function name and dataframe 
    for col in df.columns:    # for every loop through a column, store columnn name into col 
        nan_count = df[col].isna().sum() # sweep for nan values and sum the amount, then store as object
        if nan_count > 0:                    # if nan values are greater than zero.... flag it 
            print(col, nan_count)            # output column name and nan count 

count_nan(site_exposure)



hours_per_incident_rate 1
hours_zscore_4wk 1


In [ ]:
# I will be leaving the two defined columns with nan values alone, seeing as week 1 had no incidents
# This in business terms means week 1 was the safest week out of the year. 


In [63]:
#                            --- Exploratory Data Analysis ---

# Let's build out a threshold for incident per 10,000 hours 
# Any incidents that occur outisde the expected range of distribution is considered high risk 

q90 = site_exposure['incidents_per_10k_hours_rate'].quantile(0.90)
q95 = site_exposure['incidents_per_10k_hours_rate'].quantile(0.95)

print(f'Incidents outside of the 90th percentile {q90}')
print(f'Incidents outside of the 95th percentile {q95}')


Incidents outside of the 90th percentile 2.0410470786438415
Incidents outside of the 95th percentile 2.5689784329992955


In [68]:
# Creating a 90th and 95th percentile risk flag and store output as an integer

site_exposure['high_risk'] = (site_exposure['incidents_per_10k_hours_rate'] >= q90).astype(int)

site_exposure['extreme_risk'] = (site_exposure['incidents_per_10k_hours_rate'] >= q95).astype(int)

site_exposure[['incidents_per_10k_hours_rate', 'high_risk', 'extreme_risk']] 

,incidents_per_10k_hours_rate,high_risk,extreme_risk
0,0.971352,0,0
1,1.176319,0,0
2,1.092521,0,0
3,0.793025,0,0
4,1.463800,0,0
5,1.371483,0,0
6,0.363852,0,0
7,1.104876,0,0
8,0.346212,0,0
9,0.729548,0,0
